# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [7]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
import joblib



## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [8]:
ws = Workspace.from_config()
experiment_name = 'capstone-project'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: udacitymlworkspace
Azure region: eastus
Subscription id: dce3a2e3-2f87-4a43-9fd7-d3a4bdeb704f
Resource group: udacityml


In [9]:
cluster_name = "udacityMLCluster"

# Create compute cluster

#verify that cluster does not exist
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_name = os.environ.get('AML_COMPUTE_CLUSTER_NAME', cluster_name)
    compute_min_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MIN_NODES', 0)
    compute_max_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MIN_NODES', 4)

    vm_size = os.environ.get('AML_COMPUTE_CLUSTER_SKU', 'Standard_D2_V2')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size, min_nodes= compute_min_nodes, max_nodes=compute_max_nodes)

    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

compute_target.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [10]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1, evaluation_interval=2, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
        "C": uniform(0.5, 1.0),
        "max_iter": choice([5,100])
    }
)

if "training2" not in os.listdir():
    os.mkdir("./training2")


# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
 
src = ScriptRunConfig(source_directory='./capstone',
                            script='train.py',
                            compute_target=compute_target,
                            environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling = param_sampling,
                                     policy=early_termination_policy,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=4,
                                     max_concurrent_runs=4)

In [11]:
#TODO: Submit your experiment
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = experiment.submit(config=hyperdrive_config)
hyperdrive_run.get_status()
#RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_917cc7de-2599-48c1-8b70-e5ee94633f35
Web View: https://ml.azure.com/runs/HD_917cc7de-2599-48c1-8b70-e5ee94633f35?wsid=/subscriptions/dce3a2e3-2f87-4a43-9fd7-d3a4bdeb704f/resourcegroups/udacityml/workspaces/udacitymlworkspace&tid=8c9a0d04-e250-4aeb-85f4-1b71f173e263

Streaming azureml-logs/hyperdrive.txt

[2023-03-25T16:00:10.630910][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-03-25T16:00:11.0991641Z][SCHEDULER][INFO]Scheduling job, id='HD_917cc7de-2599-48c1-8b70-e5ee94633f35_0' 
[2023-03-25T16:00:11.2545364Z][SCHEDULER][INFO]Scheduling job, id='HD_917cc7de-2599-48c1-8b70-e5ee94633f35_1' 
[2023-03-25T16:00:11.356377][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-03-25T16:00:11.3635869Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_917cc7de-2599-48c1-8b70-e5ee94633f35_0' 
[2023-03-25T16:00:11.4826154Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_917cc7de-2599-48c1-8b70

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Execution failed. User process '/azureml-envs/azureml_7eec2c8971b9410f92147a7e257297e7/bin/python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error: Traceback (most recent call last):\n  File \"<string>\", line 197, in <module>\n  File \"<string>\", line 193, in main\n  File \"/azureml-envs/azureml_7eec2c8971b9410f92147a7e257297e7/lib/python3.6/runpy.py\", line 261, in run_path\n    code, fname = _get_code_from_file(run_name, path_name)\n  File \"/azureml-envs/azureml_7eec2c8971b9410f92147a7e257297e7/lib/python3.6/runpy.py\", line 231, in _get_code_from_file\n    with open(fname, \"rb\") as f:\nFileNotFoundError: [Errno 2] No such file or directory: 'train.py'\n\n",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorRes

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_metrics())
print(best_run.get_file_names())

In [ ]:
#TODO: Save the best model
model = best_run.register_model(model_name='model_hyperdrive', model_path='outputs/model.joblib')

best_run.download_file(best_run.get_file_names()[3], output_file_path='./outputs/')

joblib.load('./outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

